In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
import os

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
def process_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) 
    return image

In [5]:
def predict_image(image_path, model):
    model.eval()  
    image = process_image(image_path)  
    outputs = model(image)  
    _, predicted = torch.max(outputs, 1)  
    predicted_class = class_names[predicted.item()]  
    return predicted_class, outputs

In [6]:
def show_image_and_prediction(image_path, model):
    print(f"Girdiğiniz Dosya Yolu: {image_path}")
    if not os.path.exists(image_path):
        print(f"Dosya bulunamadı: {image_path}")
        return
    
    predicted_class, outputs = predict_image(image_path, model)
    _, predicted = torch.max(outputs, 1)
    confidence = F.softmax(outputs, dim=1)[0][predicted.item()] * 100 
    
    print(f"Attığınız Resim: {image_path}")
    print(f"Uçak Modeli: {predicted_class}")
    print(f"Doğruluk Oranı: {confidence.item():.2f}%")
    
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')  
    plt.show()

In [7]:
data_dir = './data/classes'

In [8]:
dataset = ImageFolder(data_dir, transform=transform)

In [9]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size  

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [10]:
class_names = ['A10','A400M','AG600','AH64','An22','An72','An124','An225','AV8B','B1']

In [11]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5) 
        self.pool = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(16, 32, 5) 
        self.fc1 = nn.Linear(32 * 53 * 53, 256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10)
            
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001 , momentum = 0.9)

In [13]:
for epoch in range(1):
    print(f'deniyorum epoch {epoch} ... ')
    running_loss = 0.0 
    
    for i, data in enumerate(train_loader):
        
        inputs , labels = data
    
        optimizer.zero_grad()
    
        outputs = net(inputs)
        
        loss = loss_function(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'Loss:{running_loss / len(train_loader):.4f}')

deniyorum epoch 0 ... 
Loss:2.1883


In [14]:
torch.save(net.state_dict(), 'trained_net.pth')

In [15]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\emreg\AppData\Local\Temp\ipykernel_25800\1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [16]:
image_path = 'data/test/0a89565a88041220e21da1029da7996e_0.jpg'

In [ ]:
show_image_and_prediction(image_path, net)

Girdiğiniz Dosya Yolu: data/test/0a89565a88041220e21da1029da7996e_0.jpg
Attığınız Resim: data/test/0a89565a88041220e21da1029da7996e_0.jpg
Uçak Modeli: A10
Doğruluk Oranı: 20.89%
